In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import torch
import torch.nn as nn
import torchvision
import torch.optim as optim
from torch.utils.data import DataLoader,random_split,Dataset, ConcatDataset  
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

from torchinfo import summary
from tqdm import tqdm

In [ ]:
seed = 129
# random.seed(seed)
# np.random.seed(seed)
torch.manual_seed(seed)

torch.backends.cudnn.deterministic = True


num_epochs = 10
batch_size = 32
device = 'cuda'

In [ ]:
train_dataset_whole = torchvision.datasets.CIFAR10('./data', train=True, download=True, transform=transforms.ToTensor())
test_dataset = torchvision.datasets.CIFAR10('./data', train=False, download=True, transform=transforms.ToTensor())

train_dataset, valid_dataset = random_split(train_dataset_whole,[int(len(train_dataset_whole)*0.9), int(len(train_dataset_whole)*0.1)],
                                  generator=torch.Generator().manual_seed(seed))

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=10, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, num_workers=10, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=10, shuffle=False)

In [ ]:
inputs, labels = next(iter(train_loader))
out = torchvision.utils.make_grid(inputs)

plt.imshow(out.numpy().transpose(1,2,0))

In [ ]:
model = torchvision.models.resnet18(pretrained=True).to('cuda')
model.fc = torch.nn.Linear(in_features=512, 
                    out_features=10, # same number of output units as our number of classes
                    bias=True).to('cuda')

In [ ]:
summary(model=model, 
        input_size=(32, 3, 32, 32),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"]
)

In [ ]:
for name, param in model.named_parameters():
    if name[5] < '2': 
        param.requires_grad = False

In [ ]:
optimizer = torch.optim.SGD(model.parameters(),lr=0.0001)
criterion = nn.CrossEntropyLoss()

In [ ]:
def train_model(model, train_loader, criterion, optimizer, device, epochs):
    model.train()
    train_loss, total_samples, total_correct = 0,0,0
    
    for i,(imgs, labels) in enumerate(tqdm(train_loader)):
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * imgs.size(0)
        _, predicted = torch.max(outputs.data,1)

        total_correct += (predicted == labels).sum().item()
        total_samples += labels.size(0)
        accuracy = (total_correct/total_samples)*100

    print("Trainining Epoch: [{}]  loss: [{:.2f}] Accuracy :[{:.2f}]".format(epochs+1, train_loss/len(train_loader), accuracy))

    return train_loss/len(train_loader), accuracy

In [ ]:
def test_model(model, test_loader, criterion,  device, epochs):
    model.eval()
    test_loss, total_samples, total_correct = 0,0,0
    with torch.no_grad():
        for i,(imgs, labels) in enumerate(tqdm(test_loader)):
            imgs, labels = imgs.to(device), labels.to(device)

            outputs = model(imgs)
            loss = criterion(outputs, labels)

            test_loss += loss.item() * imgs.size(0)
            _, predicted = torch.max(outputs.data,1)

            total_correct += (predicted == labels).sum().item()
            total_samples += labels.size(0)
            accuracy = (total_correct/total_samples)*100

    print("Testing Epoch: [{}]  loss: [{:.2f}] Accuracy :[{:.2f}]".format(epochs+1, test_loss/len(test_loader), accuracy))

    return test_loss/len(test_loader), accuracy

In [ ]:
history = {'train_loss': [], 'valid_loss': [],'train_acc':[],'valid_acc':[]}

for i in range(num_epochs):
    train_loss, train_acc = train_model(model, train_loader, criterion, optimizer, device, i)
    validation_loss, validation_acc = test_model(model, valid_loader, criterion, device, i)
    
    history['train_loss'].append(train_loss)
    history['valid_loss'].append(validation_loss)
    history['train_acc'].append(train_acc)
    history['valid_acc'].append(validation_acc)


In [ ]:
_,_ = test_model(model, test_loader, criterion, device, i)

In [ ]:
45.80/62.01

In [ ]:
plt.figure(figsize=(8,5))
# plt.title('Without Pretraining')
plt.subplot(121)
plt.plot(history['train_loss'],label='Training Loss')
plt.plot(history['valid_loss'],label='Validation Loss')
plt.legend()
plt.xlabel('Epcohs')
plt.ylabel('Loss')
plt.subplot(122)
plt.plot(history['train_acc'],label='Training Accuracy')
plt.plot(history['valid_acc'],label='Validation Accuracy')
plt.legend()
plt.xlabel('Epcohs')
plt.ylabel('Accuracy')
plt.show()